In [ ]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import geopandas as gpd
import momepy
import os

import shapely

In [ ]:
local_crs = 3414

place = "Singapore"

study_area = gpd.read_file("./Singapore_studyArea.shp").to_crs(epsg=4326)

study_area_old = study_area

# assuming your geodataframe is called `gdf` and the geometry column is called `geometry`
study_area = study_area.geometry.unary_union

In [ ]:
osm_graph= ox.graph.graph_from_polygon(study_area, network_type='drive')
osm_graph = ox.projection.project_graph(osm_graph, to_crs=local_crs)

streets = ox.consolidate_intersections(osm_graph, rebuild_graph=True, tolerance=15, dead_ends=False)

streets = ox.graph_to_gdfs(
    osm_graph,
    nodes=False,
    edges=True,
    node_geometry=False,
    fill_edge_geometry=True
)
streets.head()

In [ ]:
osm_graph= ox.graph.graph_from_polygon(study_area, custom_filter='["railway"~"rail|subway|narrow_gauge|monorail"]')
osm_graph = ox.projection.project_graph(osm_graph, to_crs=local_crs)

rails = ox.consolidate_intersections(osm_graph, rebuild_graph=True, tolerance=15, dead_ends=False)

rails = ox.graph_to_gdfs(
    osm_graph,
    nodes=False,
    edges=True,
    node_geometry=False,
    fill_edge_geometry=True
)
rails.head()

In [ ]:
waters = []

# Get the water bodies as geometries
water_bodies = ox.geometries_from_polygon(study_area, tags={'natural': 'water'}).set_crs("epsg: 4326").to_crs(local_crs)

# Explode the GeoDataFrame into a GeoSeries of polygons and multipolygons
gs = water_bodies.explode()

# Convert each polygon into a MultiLineString
mls = gs.geometry.boundary

# Convert the MultiLineString into a DataFrame
water_bodies = gpd.GeoDataFrame({'geometry': mls})

# Add a column to specify the polygon ID
water_bodies['polygon_id'] = gs.index

print(water_bodies)

for water_body in water_bodies.geometry:
    # Create a LineString object from the coordinates
    waters.append(water_body)

coastlines = ox.geometries_from_polygon(study_area, tags={'natural': 'coastline'}).set_crs("epsg: 4326").to_crs(local_crs)

coastlines = coastlines[coastlines.geom_type == "LineString"].reset_index(drop=True)

for coastline in coastlines.geometry:
    # Create a LineString object from the coordinates
    waters.append(coastline)

# # Create a GeoDataFrame of the water area boundaries
waters = gpd.GeoDataFrame(geometry=waters)
waters = waters[["geometry"]]
waters.head()

In [ ]:
ax = streets.plot(figsize=(16, 16))
rails.plot(ax=ax, color='r')
waters.plot(ax=ax, color='k')

In [ ]:
enclosures = momepy.enclosures(streets, limit=gpd.GeoSeries([study_area]), additional_barriers=[streets, rails, waters])

In [ ]:
enclosure_raw = enclosures

# collapse columns into a single column
collapsed_series = enclosures.apply(lambda row: row[1], axis=1)

In [ ]:
print(collapsed_series)

In [ ]:
enclosures.shape# create output directory if it doesn't exist
output_dir = './out'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
gpd.GeoDataFrame(geometry=collapsed_series, crs=local_crs).to_parquet('./out/enclosures.pq')

In [ ]:
# osm_graph= ox.graph.graph_from_polygon(study_area, custom_filter='["highway"~"motorway"]')
# osm_graph = ox.projection.project_graph(osm_graph, to_crs=local_crs)

# main_roads = ox.consolidate_intersections(osm_graph, rebuild_graph=True, tolerance=1000)

# main_roads = ox.graph_to_gdfs(
#     osm_graph,
#     nodes=False,
#     edges=True,
#     node_geometry=False,
#     fill_edge_geometry=True
)

In [ ]:

boundaries = []

for polygon in study_area.geoms:
    # Get the boundary coordinates of the polygon
    boundary = polygon.boundary.coords[:]
    # Add the boundary coordinates to the list
    boundaries.append(shapely.geometry.LineString(boundary))

boundaries = gpd.GeoDataFrame({'geometry': boundaries}, geometry='geometry', crs=4326)

merged_gdf =  gpd.GeoDataFrame(pd.concat([boundaries.to_crs(local_crs).geometry, main_roads.geometry]), crs=local_crs)

In [ ]:
merged_lines = shapely.ops.linemerge(list(merged_gdf['geometry']))
border_lines = shapely.ops.unary_union(merged_lines)
decomposition = shapely.ops.polygonize(border_lines)

In [ ]:
# Create a list of tuples with the geometry and any additional attributes
records = [(geometry,) for geometry in decomposition]

# Create a new GeoDataFrame from the list of tuples
chunks = gpd.GeoDataFrame.from_records(records, columns=['geometry'])

In [ ]:
print(chunks)

In [ ]:
ax = chunks.plot(figsize=(16, 16))

In [ ]:
gdf.to_parquet('./Singapore/local_authorities.pq')